In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

pd.set_option('display.max_rows',28*28)
pd.set_option('display.max_columns',28*28)


# Setup

## Data loading, preprocessing, and exploration


In [ ]:
# Load the MNIST dataset training and test sets as numpy arrays

#renamed X_train_full and y_train_full for consistency
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
assert X_train.shape == (60000, 28, 28)
assert X_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)


## Data Inspection

In [ ]:
print("Number of samples in training data:", X_train.shape[0])
print('Classes in training data:',np.unique(y_train))
print("Dimensions of training data:",(X_train.shape[1],X_train.shape[2]))
print("Max and min pixel values:", (X_train.max(), X_train.min()))


In [ ]:
print('Classes' ,np.unique(y_train, return_counts=True)[0])
print('Class distribution' ,np.unique(y_train, return_counts=True)[1])


In [ ]:
print("Number of samples in test data:", X_test.shape[0])
print('Classes in test data:',np.unique(y_test))
print("Dimensions of test data:",(X_test.shape[1],X_test.shape[2]))
print("Max and min pixel values:", (X_test.max(), X_test.min()))


In [ ]:
#ploting first 40 training samples 
class_dic = {k:v for k,v in enumerate(np.unique(y_train))}

def plot_examples(data = X_train, labels = y_train, n_rows=4, n_cols=10):

    plt.figure(figsize=(n_cols * 1.2, n_rows * 1.5))
    
    for row in range(n_rows):
        for col in range(n_cols):
            
            # Get next index of image
            index = n_cols * row + col
            
            # Plot the image at appropriate place in grid
            plt.subplot(n_rows, n_cols, index + 1)
            plt.imshow(data[index], cmap="binary")
            plt.title(class_dic[labels[index]], fontsize = 15)
            plt.axis('off')
            
    plt.show()

plot_examples()


## Data Preprocessing

In [ ]:
X_train = X_train/255
X_train = X_train.astype(float)

X_test = X_test/255
X_test = X_test.astype(float)

In [ ]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

## Dividing the dataset - each client has data from two classes

In [ ]:
def splitClientX():
    X_train_clients = {}
    for i in range(10):
        idx = y_train == i
        X_train_clients[i] = X_train[idx]
    return X_train_clients

def splitClienty():
    y_train_clients = {}
    for i in range(10):
        idx = y_train == i
        y_train_clients[i] = y_train[idx]
    return y_train_clients

In [ ]:
X_train_divided = splitClientX()
y_train_divided = splitClienty()

In [ ]:
#Creating client X data dictionary
X_train_clients = {}
firstHalf0 = X_train_divided[0][:len(X_train_divided[0])//2]
secondHalf0 = X_train_divided[9][len(X_train_divided[9])//2:]
X_train_clients[0] = np.concatenate((firstHalf0,secondHalf0))

for i in range(1, 10):
    firstHalf = X_train_divided[i][:len(X_train_divided[i])//2]
    secondHalf = X_train_divided[i-1][len(X_train_divided[i-1])//2:]
    X_train_clients[i] = np.concatenate((firstHalf,secondHalf))


# Checking if total training samples = 60000
n = 0
for i in X_train_clients:
    print(X_train_clients[i].shape)
    n+=len(X_train_clients[i])
print("Total training samples:", n)

In [ ]:
#Creating client y data dictionary
y_train_clients = {}
firstHalf0y = y_train_divided[0][:len(y_train_divided[0])//2]
secondHalf0y = y_train_divided[9][len(y_train_divided[9])//2:]
y_train_clients[0] = np.concatenate((firstHalf0y,secondHalf0y))

for i in range(1, 10):
    firstHalf = y_train_divided[i][:len(y_train_divided[i])//2]
    secondHalf = y_train_divided[i-1][len(y_train_divided[i-1])//2:]
    y_train_clients[i] = np.concatenate((firstHalf,secondHalf))


# Checking client class distribution
n = 0
for i in y_train_clients:
    print(f'Client {n} class distribution:', np.unique(y_train_clients[i]))
    n+=1

In [ ]:
# Printing client image with label
def printClient(client_number):
    
    for i in range(len(X_train_clients[client_number])):
        plt.figure()
        plt.imshow(X_train_clients[client_number][i],cmap = 'binary')
        plt.title(y_train_clients[client_number][i],fontsize = 15)
        plt.axis('off')
        plt.show()
        


## CNN model setup

In [ ]:
#transforming shape of data for CNN
X_train_cnn = np.expand_dims(X_train, -1)
X_train_clients_cnn = {k:np.expand_dims(v, -1) for (k,v) in X_train_clients.items()}

for client in X_train_clients_cnn:
    print('Shape of client {} data: '.format(client), X_train_clients_cnn[client].shape)

In [ ]:
tf.random.set_seed(1)
keras.backend.clear_session()


# Defining functions

## Initialising CNN models with learning rate

In [ ]:

def initialiseCNN(lr= 0.125):
    model = keras.Sequential([
    
    keras.Input(shape=(28, 28, 1)),
    
    keras.layers.Conv2D(32, kernel_size=(5, 5), padding = 'same', activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2), padding = 'same'),
    
    keras.layers.Conv2D(64, kernel_size=(5, 5), padding = 'same',activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2), padding = 'same'),
    
    keras.layers.Flatten(),
    keras.layers.Dense(units=512, activation='relu'),
    keras.layers.Dense(10, activation="softmax"),
])
    
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.SGD(learning_rate=lr),
              metrics=['accuracy'])
    
    return model

## Calculating model accuracy

In [ ]:
# examine accuracy before training
def modelAccuracy(model):
    prob = model.predict(X_test)
    pred = np.argmax(prob, axis=-1)

    print("Model can predict classes: ", np.unique(pred))
    print("Model accuracy: {}".format(accuracy_score(y_test, pred)),
          "\n-------------------------------------------------------------")
    
    return accuracy_score(y_test, pred)

## Defining train client function

In [ ]:
def trainClient(client_number, models, batch_size = 10, epochs = 5):

    print("Training client ", str(client_number))
    models[client_number].fit(X_train_clients_cnn[client_number], y_train_clients[client_number], 
                                     batch_size=batch_size,epochs=epochs)

## Defining the aggregation function

In [ ]:
def aggregateModel(models, global_model, client_importance):
    client_model_weights = []
    for client_number in participatingClients:
        client_model_weights.append(np.array(client_models[client_number].get_weights(), dtype = object))
    
    client_model_weighted_weights = [x*y for x,y in zip(client_model_weights,client_importance)]
    global_weights = sum(client_model_weighted_weights)
    
    for i in range(10):
        models[i].set_weights(global_weights)
        
    global_model.set_weights(global_weights)



##  C parameter

In [ ]:
import random

def selectClients(C=0.5):
    participatingClients = []
    clientPerRound = int(10*C)
    while len(participatingClients) <clientPerRound:
        newClient = random.randrange(10)
        if newClient not in participatingClients:
            participatingClients.append(newClient)
    return participatingClients



## Assigning client importance

In [ ]:
def calculateClientImportance(participatingClients):
    client_data_size = [len(X_train_clients[k]) for k in participatingClients]
    client_importance = [x/sum(client_data_size) for x in client_data_size]
    for x in zip(participatingClients, client_importance):
        print("Client {} importance is {:.4f}".format(x[0], x[1]))
    return client_importance

# <font color = 'red'>Training</font>

In [ ]:
number_of_rounds = 100   #set to desired value
C = 0.5                  #set to desired value
lr = 0.125               #set to desired value

global_model = initialiseCNN()

client_models = {}
for i in range(10):
    client_models[i] = initialiseCNN(lr=lr) 
    
global_accuracy = []

for n in range(1, 1+number_of_rounds):
    print('Communication round #{}'.format(n))
    
    participatingClients = selectClients(C)
    print(f"Clients participating in round {n}: ", participatingClients)

    for i in participatingClients:
        trainClient(i, models = client_models, epochs = 5)    
    print('Client {} trained for round #{}'.format(i, n))
    
    clientImportance = calculateClientImportance(participatingClients)

    aggregateModel(client_models, global_model, clientImportance)
    print('Model aggregated for round #{}'.format(n))

    print("Global model accuracy: ", modelAccuracy(global_model))
    global_accuracy.append(modelAccuracy(global_model))

In [ ]:
# saving accuracy to csv

import csv
 
data =[[x] for x in global_accuracy]

file = open('accuracy_federated_learning_10clients.csv', 'w+', newline ='')
 
with file:
    write = csv.writer(file)
    write.writerows(data)